# Deep Markov Chains

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


The goal of this notebook is to implement a recommendation method based on $k$ neural networks and an underlying Markov chain with memory.

In [ ]:
from keras.layers import Dense, Activation, Input, Embedding
from keras.models import load_model, Model
from keras.utils import Sequence
from keras.backend import sum

import matplotlib.pyplot as plt
import csv
import math
import numpy as np
import random

## Architecture

In [ ]:
# Nombre de valeurs dans le dictionnaire + la valeur vide
d = 14370 + 1 # 1 catégories supplémentaires : une <EOS>
# Taille de l'input
Tx = 64
# Taille de l'output
Ty = 16
# Batch Size
m = 8192
# Dimension de l'embedding
n_e = 1024

l = [1, 2, 3, 4, 8, 16, 32, 64]

The architecture aims to be as simple as possible. We start by passing the $k$ inputs through the embedding of dimension n_e before summing them and passing them through a Softmax layer.

In [ ]:
def getDeepMarkovChain(k = 2):
  inpt = Input(shape=(k))
  x = Embedding(d , n_e)(inpt)
  x = sum(x , axis = 1, keepdims=False)
  outpt = Dense(d , activation="softmax")(x)
  model = Model(inputs = inpt, outputs = outpt)
  return model


In [ ]:
model = getDeepMarkovChain(k=64)
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics= "accuracy")
model.summary()
model.save("/content/drive/MyDrive/PSC Recommandation séquentielle/Modèles/Chaînes de Markov/DeepChain/Model64")

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 64, 1024)          14715904  
_________________________________________________________________
tf.math.reduce_sum (TFOpLamb (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 14371)             14730275  
Total params: 29,446,179
Trainable params: 29,446,179
Non-trainable params: 0
_________________________________________________________________
INFO:tensorflow:Assets written to: /content/drive/MyDrive/PSC Recommandation séquentielle/Modèles/Chaînes de Markov/DeepChain/Model64/assets


In [ ]:
class DataGenerator(Sequence):
  def __init__(self , nb_lines, X_path, Y_path , k):
    self.X_path = X_path
    self.X_reader = csv.reader(open(folder + X_path , "r"))
    self.Y_path = Y_path
    self.Y_reader = csv.reader(open(folder + Y_path , "r"))
    self.nb_lines = nb_lines
    self.k = k

  def __len__(self):
    return math.ceil(self.nb_lines/m)

  def __getitem__(self, idx):
    X1 = []
    Y = []
    for i in range(m):
      row = self.getNextSample()
      r = random.randrange(0 , Tx+Ty-self.k)
      x = [int(j) for j in row[r:r+self.k]]
      y = [int(row[r+self.k])]
      X1.append(x)
      Y.append(y)
    X1 = np.array(X1)
    return np.array(X1) , np.array(Y)

  def getNextSample(self):
    x = next(self.X_reader , None)
    y = next(self.Y_reader , None)
    if x is None:
      self.X_reader = csv.reader(open(folder + self.X_path , "r"))
      self.Y_reader = csv.reader(open(folder + self.Y_path , "r"))
      x = next(self.X_reader , None)
      y = next(self.Y_reader , None)
    return x+y

In [ ]:
for k in l:
  model = load_model("/content/drive/MyDrive/PSC Recommandation séquentielle/Modèles/Chaînes de Markov/DeepChain/Model" + str(k))
  model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics= "accuracy")
  train_gen = DataGenerator(3705954
                          , "s_X_train.csv"
                          ,"s_Y_train.csv" ,
                          k)
  model.fit(train_gen , epochs=5, verbose = 1)
  model.save("/content/drive/MyDrive/PSC Recommandation séquentielle/Modèles/Chaînes de Markov/DeepChain/Model" + str(k))


In [ ]:
train_gen = DataGenerator(3705954
                          , "s_X_train.csv"
                          ,"s_Y_train.csv" ,
                          64)
model.fit(train_gen , epochs=10, verbose = 1)

model.save("/content/drive/MyDrive/PSC Recommandation séquentielle/Modèles/Chaînes de Markov/DeepChain/Model64")

## Prediction

In [ ]:
ModelTable = {}
def loadModel(l):
  modelpath = "/content/drive/MyDrive/PSC Recommandation séquentielle/Modèles/Chaînes de Markov/DeepChain/Model"
  for i in l:
    ModelTable[i] = load_model(modelpath + str(i))


In [ ]:
def SimplePred(X , k):
  x = X[len(X) - k : len(X)]
  y = []
  for i in range(Ty):
    yhat = ModelTable[k](np.reshape(np.array(x) , (1 , k)))
    i = np.argmax(yhat)
    x.append(i)
    y.append(i)
    x = x[1:len(x)]
  return y

def verySimplePredict(X , k):
  x = X[len(X) - k : len(X)]
  yhat = np.array(ModelTable[k](np.reshape(np.array(x) , (1 , k))))[0][:]
  y = []
  while len(y) < 16:
    j = np.argmax(yhat)
    yhat[j] = 0
    if j != 0:
      y.append(j)

  return y


In [ ]:
with open(folder + "Y_dev.csv" , "r") as csvfile:
  r = csv.reader(csvfile)
  for i in range(0) : next(r)
  T = [int(i) for i in next(r)]
  print("Séquence de référence: ")
  print(T)

for k in l:
  with open(folder + "X_dev.csv" , "r") as csvfile:
    r = csv.reader(csvfile)
    for i in range(0) : next(r)
    seq = next(r)
    T = SimplePred([int(i) for i in seq] , k)
    print("Séquence inférée pour k = " + str(k) + " :")
    print(T)
    T = verySimplePredict([int(i) for i in seq] , k)
    print("k meilleurs candidats pour k = " + str(k) + " :")
    print(T)

In [ ]:
def sumPred(X , l):
  x = X
  y = []
  for i in range(Ty):
    yhat = np.zeros((1 , d))
    for k in l:
      yhat += ModelTable[k](np.reshape(np.array(x[len(x) - k : len(x)]) , (1 , k)))
    i = np.argmax(yhat)
    x.append(i)
    y.append(i)
  return y

In [ ]:
with open(folder + "Y_dev.csv" , "r") as csvfile:
  r = csv.reader(csvfile)
  for i in range(0) : next(r)
  T = [int(i) for i in next(r)]
  print("Séquence de référence: ")
  print(T)

with open(folder + "X_dev.csv" , "r") as csvfile:
  r = csv.reader(csvfile)
  for i in range(0) : next(r)
  seq = next(r)
  T = sumPred([int(i) for i in seq] , l)
  print("Séquence inférée pour avec la somme des estimateurs :")
  print(T)

In [ ]:
def verySimpleSumPredict(X , l):
  x = X
  yhat = np.zeros((d))
  for k in l:
    yhat += np.array(ModelTable[k](np.reshape(np.array(x[len(x)-k : len(x)]) , (1 , k))))[0][:]
  y = []
  while len(y) < 16:
    j = np.argmax(yhat)
    yhat[j] = 0
    if j != 0:
      y.append(j)

  return y

In [ ]:
with open(folder + "Y_dev.csv" , "r") as csvfile:
  r = csv.reader(csvfile)
  for i in range(0) : next(r)
  T = [int(i) for i in next(r)]
  print("Séquence de référence: ")
  print(T)

with open(folder + "X_dev.csv" , "r") as csvfile:
  r = csv.reader(csvfile)
  for i in range(0) : next(r)
  seq = next(r)
  T = verySimpleSumPredict([int(i) for i in seq] , l)
  print("k meilleurs pour avec la somme des estimateurs :")
  print(T)

In [ ]:
def predWithoutRedundance(X , k):
  x = X[len(X) - k : len(X)]
  y = []
  for i in range(Ty):
    yhat = np.array(ModelTable[k](np.reshape(np.array(x) , (1 , k))))
    i = np.argmax(yhat)
    while (i in y or i == 0):
      yhat[0][i] = 0
      i = np.argmax(yhat)
    x.append(i)
    y.append(i)
    x = x[1:len(x)]
  return y

In [ ]:
with open(folder + "Y_dev.csv" , "r") as csvfile:
  r = csv.reader(csvfile)
  for i in range(0) : next(r)
  T = [int(i) for i in next(r)]
  print("Séquence de référence: ")
  print(T)

for k in l:
  with open(folder + "X_dev.csv" , "r") as csvfile:
    r = csv.reader(csvfile)
    for i in range(0) : next(r)
    seq = next(r)
    T = predWithoutRedundance([int(i) for i in seq] , k)
    print("Séquence inférée sans redondance pour k = " + str(k) + " :")
    print(T)

In [ ]:
def sumPredWithoutRedundance(X , l):
  x = X
  y = []
  for i in range(Ty):
    yhat = np.zeros((1,d))
    for k in l:
      yhat += np.array(ModelTable[k](np.reshape(np.array(x[len(x) - k : len(x)]) , (1 , k))))
    i = np.argmax(yhat)
    while (i in y or i == 0):
      yhat[0][i] = 0
      i = np.argmax(yhat)
    x.append(i)
    y.append(i)
    x = x[1:len(x)]
  return y

In [ ]:
with open(folder + "Y_dev.csv" , "r") as csvfile:
  r = csv.reader(csvfile)
  for i in range(0) : next(r)
  T = [int(i) for i in next(r)]
  print("Séquence de référence: ")
  print(T)

with open(folder + "X_dev.csv" , "r") as csvfile:
  r = csv.reader(csvfile)
  for i in range(0) : next(r)
  seq = next(r)
  T = sumPredWithoutRedundance([int(i) for i in seq] , l)
  print("Séquence inférée sans redondance pour avec la somme des estimateurs :")
  print(T)

## Comparing the accuracies of the different methods

In [ ]:
def getUnorderedAccuracy(method):
  with open(folder + "s_X_dev.csv" , "r") as csvfile:
    r1 = csv.reader(csvfile)
    r2 = csv.reader(open(folder + "s_Y_dev.csv" , "r"))
    counteq = 0
    nlines = 0
    for i in range(1000):
      nlines += 1
      y = [int(i) for i in next(r2)]
      y_hat = method([int(i) for i in next(r1)])
      for i in range(16):
        if y[i] in y_hat:
          counteq += 1
  return (counteq/((nlines)*16))

In [ ]:
for k in l:
  acc = getUnorderedAccuracy(lambda x : predWithoutRedundance(x , k))
  print("Accuracy of prediction without redundance for k =" + str(k) , " :")
  print(acc)

In [ ]:
kbestcandacc = {}
for k in l:
  kbestcandacc[k] = getUnorderedAccuracy(lambda x : verySimplePredict(x , k))
  print("Accuracy of k best candidates for k =" + str(k) , " :")
  print(kbestcandacc[k])

In [ ]:
acc = getUnorderedAccuracy(lambda x : sumPredWithoutRedundance(x , l))
print("Accuracy pour la somme des estimateurs :")
print(acc)

In [ ]:
acc = getUnorderedAccuracy(lambda x : verySimpleSumPredict(x , l))
print("Accuracy pour les k meilleurs candidats de la somme des estimateurs :")
print(acc)

In [ ]:
def sumKBestWithConvexWeights(X , l, w):
  x = X
  yhat = np.zeros((d))
  for k in l:
    yhat += w[k]*np.array(ModelTable[k](np.reshape(np.array(x[len(x)-k : len(x)]) , (1 , k))))[0][:]
  y = []
  while len(y) < 16:
    j = np.argmax(yhat)
    yhat[j] = 0
    if j != 0:
      y.append(j)

  return y

In [ ]:
with open(folder + "Y_dev.csv" , "r") as csvfile:
  r = csv.reader(csvfile)
  for i in range(0) : next(r)
  T = [int(i) for i in next(r)]
  print("Séquence de référence: ")
  print(T)

with open(folder + "X_dev.csv" , "r") as csvfile:
  r = csv.reader(csvfile)
  for i in range(0) : next(r)
  seq = next(r)
  T = sumKBestWithConvexWeights([int(i) for i in seq] , l , kbestcandacc)
  print("k meilleurs pour avec la somme des estimateurs :")
  print(T)

In [ ]:
acc = getUnorderedAccuracy(lambda x : sumKBestWithConvexWeights(x , l, kbestcandacc))
print("Accuracy pour les k meilleurs candidats de la somme des estimateurs :")
print(acc)

In [ ]:
def getArbitraryConvexWeights(l):
  weights = {}
  for k in l:
    weights[k] = bestweights[k] + 0.02*random.random()
  return weights